In [294]:
import random
import numpy as np
import time
import os
import json
import copy

status = []  # 存放n皇后坐标，index i表示row，对应元素值表示col，且元素值互不相同，从0到n-1
d1 = []  # 存放每条正对角线上的皇后个数，长度2n-1
d2 = []  # 存放每条负对角线上的皇后个数，长度2n-1
d2_base = 0  # 负对角线row-col范围为[-(2n-1)/2,(2n-1)/2]，转化成index需要加上base = (2n-1)/2
collisions = 0  # 冲突数

# 更新冲突数，将d1和d2中大于1的元素减1后累加,时间复杂度O(n)
def update_collisions():
    global collisions
    collisions = 0
    for i in d1:
        if i>1:
            collisions += i-1
    for j in d2:
        if j>1:
            collisions += j-1

# 判断row行上的皇后是否处在有冲突的位置，即d1和d2对应元素值是否大于1
def is_attacked(row):
    if d1[row+status[row]]>1:
        return True
    elif d2[d2_base+row-status[row]]>1:
        return True
    
    return False

# 测试调换皇后之后的collisions的变化
# 由于每个皇后只会影响两条对角线，那么两个皇后对换最多可能影响8条对角线，即d1和d2中的8个元素。
# 只需将可能影响到的元素取出而不需要遍历，时间复杂度O(1)
def swap_test(i,j):
    collisions_change = 0
    
    # 两个“源”皇后对应4条对角线上的值
    d1_i_s = d1[i+status[i]]
    d2_i_s = d2[d2_base+i-status[i]]
    d1_j_s = d1[j+status[j]]
    d2_j_s = d2[d2_base+j-status[j]]
    # 两个“目的”皇后对应4条对角线上的值
    d1_i_d = d1[i+status[j]]
    d2_i_d = d2[d2_base+i-status[j]]
    d1_j_d = d1[j+status[i]]
    d2_j_d = d2[d2_base+j-status[i]]
    
    # 取走两个“源”皇后最多会影响4条对角线
    if d1_i_s >1:
        collisions_change -= 1
    if d2_i_s >1:
        collisions_change -= 1
    if d1_j_s >1:
        collisions_change -= 1
    if d2_j_s >1:
        collisions_change -= 1
    # 有可能两个皇后在同一条正/负对角线上，此时若对应值为2，则change值多减了1，需要加1
    if i+status[i] == j+status[j] and d1_i_s == 2:
        collisions_change += 1
    if i-status[i] == j-status[j] and d2_i_s == 2:
        collisions_change += 1
    
    # 放置两个“目的”皇后最多会影响4条对角线
    if d1_i_d >=1:
        collisions_change += 1
    if d2_i_d >=1:
        collisions_change += 1
    if d1_j_d >=1:
        collisions_change += 1
    if d2_j_d >=1:
        collisions_change += 1
    # 同样，有可能两个皇后在同一条正/负对角线上，此时若对应值为0，则change值少加了1，需要回复加1
    if i+status[j] == j+status[i] and d1_i_d == 0:
        collisions_change += 1
    if i-status[j] == j-status[i] and d2_i_d == 0:
        collisions_change += 1
        
    return collisions_change

# 实行调换动作，更新d1,d2
def swap_performed(i,j):
    # 更新d1,d2
    d1[i+status[i]]-=1
    d2[d2_base+i-status[i]]-=1
    d1[j+status[j]]-=1
    d2[d2_base+j-status[j]]-=1
    d1[i+status[j]]+=1
    d2[d2_base+i-status[j]]+=1
    d1[j+status[i]]+=1
    d2[d2_base+j-status[i]]+=1
    # 对换位置
    temp = status[i]
    status[i] = status[j]
    status[j] = temp

def local_search():
    global collisions
    length = len(status)
       
    # repeat
    # for all i, j: where 皇后 status[i] 或 status[j] is attacked do
        # If 交换 status[i] 与 status[j] 能减少冲突
            # Then 交换 status[i] 与 status[j]
    # until collisions = 0 或 卡在局部最优
    iter_num = 0
    while collisions >0 and iter_num <5:
        for row_i in range(length):
            for row_j in range(row_i,length):
                if is_attacked(row_i) or is_attacked(row_j):
                    collisions_change = swap_test(row_i,row_j)
                    if collisions_change < 0:
                        swap_performed(row_i,row_j)
                        # print(status)
                        collisions += collisions_change
        iter_num += 1
    update_collisions()
    print("最终冲突数："+str(collisions)+'\n')
                    
    return status

# 初始化，随机重排数组[0,1,...,n]
# 数组[0,1,...,n]能确保每一行/列上不会有冲突的皇后对
def initialize(size):
    # status = []
    global status
    status = []
    for i in range(size):
        status.append(i)
    random.shuffle(status)  # 打乱数组顺序
    
    # 初始化d1,d2,collision
    global d1,d2,d2_base
    d2_base = (2*size - 1)//2
    d1 = [0]*(2*size-1)
    d2 = [0]*(2*size-1)
    for row in range(len(status)):
        # 正对角线上的所有皇后满足其行号与列号之和为常数
        d1[row+status[row]] += 1
        # 负对角线上的所有皇后满足其行号与列号之差为常数
        d2[d2_base+row-status[row]] += 1
    update_collisions()
    print("棋盘初始状态：")
    print(status)
    print("棋盘初始冲突数：")
    print(str(collisions) + '\n')
    
    return status
                
# 求得满足冲突数为0的一个解
def Queens_Search(size, filename):
    
    # 初始化
    status = initialize(size)
    # 计时
    time_start = time.time()
    # 进行局部搜索
    status = local_search()
    # 当一个随机解不能优化且未能消除冲突时，算法将重新生成一个随机解，并进行局部搜索
    cnt = 1
    while collisions>0 :
        print("卡在局部最优，重新初始化")
        print("重新初始化次数: ",cnt)
        status = initialize(size)    # 重新初始化
        status = local_search()
        cnt += 1
        
    time_end = time.time()
    time_cost = time_end - time_start
    
    print("棋盘最终状态：")
    print(status)
    
    # 将结果写入文件并输出
    h = open(filename, 'w', encoding='utf-8')
    h.write('问题规模：' + str(size) + '\n')
    h.write('搜索时间：' + str(time_cost) + '\n')
    h.write('重新初始化次数：' + str(cnt) + '\n')
    h.write('解：' + str(status) + '\n')
    h.close()

In [295]:
dir = "qs_config.json"
while not os.path.exists(dir):
# 判断文件是否存在
    print("file doesn't exists!")
    sys.exit()
f = open(dir, encoding='utf-8')
setting = json.load(f)
f.close()

num = setting['num']
outputfile = setting['output']

Queens_Search(num,outputfile)

棋盘初始状态：
[55548, 52778, 40156, 63069, 64674, 957, 83607, 98322, 89279, 1707, 39829, 52715, 8091, 86692, 78112, 60823, 85222, 75180, 28918, 39782, 15411, 23142, 46042, 42881, 524, 20814, 14468, 25664, 32986, 35695, 76824, 92854, 9930, 55543, 35401, 81674, 14187, 84487, 94312, 40773, 86697, 45132, 87489, 44457, 56794, 10187, 32803, 21854, 12621, 5514, 93541, 45935, 79981, 85247, 39623, 69653, 10988, 71756, 79974, 32318, 15038, 42867, 24850, 18692, 72277, 4460, 38774, 41221, 90862, 73929, 68203, 58808, 21463, 10357, 79815, 34817, 21927, 14543, 74704, 50583, 76785, 52904, 66836, 54006, 99050, 39773, 75411, 15544, 76312, 6406, 53498, 40325, 16706, 52099, 28220, 83946, 95936, 71655, 30762, 60100, 54471, 57594, 3191, 10558, 89653, 45717, 57973, 75992, 1335, 66585, 93133, 95525, 70282, 70038, 19417, 18108, 52553, 19042, 77853, 49782, 70372, 81318, 55123, 50892, 71016, 36009, 57994, 90723, 87745, 70019, 59641, 55861, 11392, 83803, 54242, 58822, 90819, 95389, 22390, 7183, 57084, 58757, 34250, 911

KeyboardInterrupt: 

In [267]:
status = [7, 4, 3, 2, 5, 1, 0, 6]
print(status)

d1 = [0]*(2*8-1)
d2 = [0]*(2*8-1)
d2_base = 7
for row in range(len(status)):
    # 正对角线上的所有皇后满足其行号与列号之和为常数
    d1[row+status[row]] += 1
    # 负对角线上的所有皇后满足其行号与列号之差为常数
    d2[d2_base+row-status[row]] += 1
print("d1: ",d1)
print("d2: ",d2)
update_collisions()
print("collisions = ",collisions)

[7, 4, 3, 2, 5, 1, 0, 6]
d1:  [0, 0, 0, 0, 0, 3, 2, 1, 0, 1, 0, 0, 0, 1, 0]
d2:  [1, 0, 0, 0, 1, 0, 2, 0, 2, 0, 0, 1, 0, 1, 0]
collisions =  5


In [268]:
print(status)
length = len(status)
for row_i in range(length):
    #print("row_i")
    for row_j in range(row_i,length):
        #print("row_j")
        if is_attacked(status,row_i) or is_attacked(status,row_j):
            # print("row%d or row%d is attacked!"%(row_i,row_j))
            collisions_change = swap_test(status,row_i,row_j)
            print("collisions_change = ",collisions_change)
            if collisions_change < 0:
                print("swap row%d and row%d"%(row_i,row_j))
                swap_performed(status,row_i,row_j)
                print("d1: ",d1)
                print("d2: ",d2)
                collisions += collisions_change
                print("collisions += collisions_change:",collisions)
                print(status)
                update_collisions()
                print(collisions)

[7, 4, 3, 2, 5, 1, 0, 6]
collisions_change =  -1
swap row0 and row1
d1:  [0, 0, 0, 0, 1, 2, 2, 0, 1, 1, 0, 0, 0, 1, 0]
d2:  [0, 1, 0, 1, 0, 0, 2, 0, 2, 0, 0, 1, 0, 1, 0]
collisions += collisions_change: 4
[4, 7, 3, 2, 5, 1, 0, 6]
4
collisions_change =  -1
swap row0 and row2
d1:  [0, 0, 0, 1, 0, 1, 3, 0, 1, 1, 0, 0, 0, 1, 0]
d2:  [0, 1, 0, 0, 1, 1, 1, 0, 2, 0, 0, 1, 0, 1, 0]
collisions += collisions_change: 3
[3, 7, 4, 2, 5, 1, 0, 6]
3
collisions_change =  1
collisions_change =  1
collisions_change =  0
collisions_change =  2
collisions_change =  2
collisions_change =  1
collisions_change =  0
collisions_change =  2
collisions_change =  -1
swap row1 and row7
d1:  [0, 0, 0, 1, 0, 1, 3, 1, 0, 1, 0, 0, 0, 0, 1]
d2:  [0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0]
collisions += collisions_change: 2
[3, 6, 4, 2, 5, 1, 0, 7]
2
collisions_change =  2
collisions_change =  2
collisions_change =  2
collisions_change =  2
collisions_change =  -1
swap row2 and row6
d1:  [0, 0, 1, 1, 0, 1, 1, 1, 0, 1,